In [13]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt

def frequecy_of_max_magnitude(filename,filter_persent = 20,n =1):
    # Load the human voice audio file

    y, sr = librosa.load(filename, sr=None)
    time = np.linspace(0,len(y)/sr ,len(y))

    # Compute Fourier Transform using FFT
    N = len(y)  # Number of samples
    frequencies = np.fft.rfftfreq(N, d=1/sr)  # Frequency bins
    fft_values = np.fft.rfft(y)  # Compute FFT
    magnitude = np.abs(fft_values)  # Magnitude spectrum

    index =np.argmax(magnitude)
    ans = frequencies[index]
    
    return ans

In [14]:
print(frequecy_of_max_magnitude("Phone/Ee.mp3"))


163.67280285035628


In [15]:
def freq_to_note(freq):
    while (freq > 165):
        freq /= 2
    if freq < 80:return "X"
    if (freq < 84.5):return "E"
    elif freq < 90 :return "F"
    elif freq < 95.5 :return "F#"
    elif freq < 101 :return "G"
    elif freq < 107 :return "G#"
    elif freq < 113.5 :return "A"
    elif freq < 120.5 :return "A#"
    elif freq < 127.5 :return "B"
    elif freq < 135 :return "C"
    elif freq < 143 :return "C#"
    elif freq < 151.5 :return "D"
    elif freq < 160.5 :return "D#"
    else :return "E"
    

freq_to_note(450)

'A'

In [16]:


import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt

def mymodel(filename,filter_persent = 20,n =1):
    # Load the human voice audio file

    y, sr = librosa.load(filename, sr=None)
    time = np.linspace(0,len(y)/sr ,len(y))

    # Compute Fourier Transform using FFT
    N = len(y)  # Number of samples
    frequencies = np.fft.rfftfreq(N, d=1/sr)  # Frequency bins
    fft_values = np.fft.rfft(y)  # Compute FFT
    magnitude = np.abs(fft_values)  # Magnitude spectrum

    # print(frequencies)
    # print(magnitude)

    # plt.subplot(2,1,1)
    # plt.plot(time,y)

    # plt.title("Amplitude in time")
    # plt.show()

    max_mag = max(magnitude)
    # print(max_mag)

    new_mag = magnitude/max_mag*100
    fileter_mag = new_mag.copy()
    fileter_mag[fileter_mag < filter_persent] = 0

    # plt.subplot(2,1,2)
    # plt.plot(frequencies, fileter_mag, color='g')
    # plt.xlabel("Frequency (Hz)")
    # plt.ylabel("Magnitude")
    # plt.title("after filter")
    # plt.xlim(0, 600)  # Limit x-axis to 5 kHz (Human voice mostly < 5kHz)
    # plt.grid()
    # plt.show()
    ans = []
    for i in range(len(fileter_mag)):
        if fileter_mag[i] >= filter_persent:
            # print(frequencies[i])
            ans.append(int(frequencies[i]))
    ans = list(set(ans))
    ans.sort()
    return ans

In [17]:
freqs = mymodel("Cchord.mp3")
# freqs = mymodel("Phone/chordDv6.mp3",10)
print(freqs)
ans = {'X'}
for i in freqs:
    if i>600:continue
    ans.add(freq_to_note(i))
    if len(ans)>=4:break


# print(ans)

[37, 130, 131, 162, 163, 164, 165, 166, 167, 196, 197, 262, 263, 330, 331, 332, 392, 393, 395, 498, 524, 525, 526, 588, 589, 660, 661, 662, 786, 982, 992, 1322, 1323, 1572, 1573, 1654, 1840]


In [18]:
def tell_note(filename): 
    freqs = mymodel(filename,20)
    ans = {'X'}
    for i in freqs:
        if i>600:continue
        ans.add(freq_to_note(i))
        if len(ans)>=4:break
    # print(ans)
    return ans


In [19]:
tell_note("Comp/E.mp3")

{'B', 'E', 'G#', 'X'}

In [20]:
from itertools import permutations

# แผนที่โน้ตเป็นตัวเลข (C = 0, C# = 1, ..., B = 11)
note_to_num = {'C': 0, 'C#': 1, 'D': 2, 'D#': 3, 'E': 4, 'F': 5, 
               'F#': 6, 'G': 7, 'G#': 8, 'A': 9, 'A#': 10, 'B': 11}

# แปลงตัวเลขกลับเป็นโน้ต
num_to_note = {v: k for k, v in note_to_num.items()}

# โครงสร้างคอร์ดพื้นฐาน (intervals จากโน้ตราก)
chords = {
    "": [0, 4, 7],
    "m": [0, 3, 7]
}
# chords = {
#     "Major": [0, 4, 7],
#     "Minor": [0, 3, 7],
#     "Diminished": [0, 3, 6],
#     "Augmented": [0, 4, 8]
# }

def identify_chord(notes):
    # แปลงโน้ตเป็นตัวเลข
    nums = [note_to_num[n] for n in notes]
    
    for perm in permutations(nums):  # ลองทุกลำดับของโน้ต
        root = perm[0]
        intervals = [(n - root) % 12 for n in perm]
        
        for chord_name, pattern in chords.items():
            if intervals == pattern:
                return f"{num_to_note[root]}{chord_name}"  # แสดงชื่อคอร์ดที่แท้จริง
    
    return "Unknown Chord"

# ทดสอบ
# print(identify_chord(["D", "F#", "A"]))  # D Major
# print(identify_chord(["B", "D", "F#"]))  # B Minor
# print(identify_chord(["C", "E", "G"]))   # C Major
# print(identify_chord(["G", "B", "D"]))   # G Major
# print(identify_chord(["A", "C", "E"]))   # A Minor

def note_to_chord(notes):
    note = list(notes)
    note.remove('X')
    ans = identify_chord(note)
    # print(type(note))
    return ans

# note_to_chord(tell_note("Comp/G3.mp3"))

In [21]:
note_to_chord(tell_note("Comp/C2.mp3"))

'C'

In [22]:
note_to_chord(tell_note("Comp/B.mp3"))

'B'

In [23]:
from itertools import permutations
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt

def mymodel(filename,filter_persent = 20):

    y, sr = librosa.load(filename, sr=None)

    N = len(y)  # Number of samples
    frequencies = np.fft.rfftfreq(N, d=1/sr)  # Frequency bins
    fft_values = np.fft.rfft(y)  # Compute FFT
    magnitude = np.abs(fft_values)  # Magnitude spectrum

    max_mag = max(magnitude)

    new_mag = magnitude/max_mag*100
    fileter_mag = new_mag.copy()
    fileter_mag[fileter_mag < filter_persent] = 0

    ans = []
    for i in range(len(fileter_mag)):
        if fileter_mag[i] >= filter_persent:
            # print(frequencies[i])
            ans.append(int(frequencies[i]))
    ans = list(set(ans))
    ans.sort()
    return ans

def tell_note(filename,percent = 20): 
    freqs = mymodel(filename,percent)
    ans = {'X'}
    for i in freqs:
        if i>600:continue
        ans.add(freq_to_note(i))
        if len(ans)>=4:break
    # print(ans)
    return ans

note_to_num = {'C': 0, 'C#': 1, 'D': 2, 'D#': 3, 'E': 4, 'F': 5, 
               'F#': 6, 'G': 7, 'G#': 8, 'A': 9, 'A#': 10, 'B': 11}
num_to_note = {v: k for k, v in note_to_num.items()}

def identify_chord(notes):
    # แปลงโน้ตเป็นตัวเลข
    nums = [note_to_num[n] for n in notes]
    
    for perm in permutations(nums):  # ลองทุกลำดับของโน้ต
        root = perm[0]
        intervals = [(n - root) % 12 for n in perm]
        
        for chord_name, pattern in chords.items():
            if intervals == pattern:
                return f"{num_to_note[root]}{chord_name}"  # แสดงชื่อคอร์ดที่แท้จริง
    
    return "Unknown Chord"

def note_to_chord(notes):
    note = list(notes)
    note.remove('X')
    ans = identify_chord(note)
    # print(type(note))
    return ans

In [24]:
filename = "Comp/B.mp3"
notes = tell_note(filename)
print(notes)

chord = note_to_chord(notes)
print(chord)

{'X', 'D#', 'B', 'F#'}
B
